# LAB 3: Creating a simple AI Agent with Langchain

## Initiation

In [ ]:
# Run this cell if you have not installed langchain in your environment
!pip install langchain
!pip install langchain-openai

In [4]:
# loading environment variables 
from dotenv import load_dotenv
load_dotenv(override=True)  # take environment variables

True

In [ ]:
# Explain how to generate apikey from Tavily dashboard
TAVILY_API_KEY = "<insert your tavily key here>"

In [3]:
# Initiating Langchain Chat Models
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4.1-mini", model_provider= "openai")


In [6]:
# Since we need a tool from langchain community, we need to install the package first
!pip install langchain-community

  Obtaining dependency information for langchain-community from https://files.pythonhosted.org/packages/03/a7/b779146b33e1f2b5ef6d44525a8cb476f8d156e2e98a251588f467d74ce3/langchain_community-0.3.23-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.3 from https://files.pythonhosted.org/packages/78/dc/5f3c0d27c91abf0bb5d103e9c9b0ff059f60cf6031a5f06f456c90731f42/aiohttp-3.11.18-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/c3/be/d0d44e092656fe7a06b55e6103cbce807cdbdee17884a5367c68c9860853/dataclasses_json-0.6.7-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic-settings<3.0.0,>=2.4.0 from https://files.pythonhosted.org/packages/b6/5f/d6d641b490fd3ec2c4c13b4244d68deea3a1b970a97be64f34fb5504ff72/pydantic_settings-2.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for httpx-sse<1.0.0,>=0.4.0 from https://files.

## Tool Calling with Langchain

In [ ]:
# Tool 1: Google Search
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("langchain tutorial")

[{'title': 'How to Build LLM Applications with LangChain Tutorial - DataCamp',
  'url': 'https://www.datacamp.com/tutorial/how-to-build-llm-applications-with-langchain',
  'content': "tutorial ### Introduction to LangChain for Data Engineering & Data Applications\nLangChain is a framework for including AI from large language models inside data pipelines and applications. This tutorial provides an overview of what you can do with LangChain, including the problems that LangChain solves and examples of data use cases. [...] Learn to build AI applications using the OpenAI API.\nStart Upskilling For Free\nIf you're captivated by the transformative powers of Generative AI and LLMs, this tutorial is perfect for you. Here, we explore LangChain - An open-source Python framework for building applications based on Large Language Models such as GPT. [...] LangChain is an open-source framework designed to facilitate the development of applications powered by large language models (LLMs). It offers 

In [41]:
# Tool 2: Weather
import requests
from langchain_core.tools import tool

@tool(parse_docstring=True)
def get_weather(latitude, longitude):
    """
        Use this tool to get current temperature and wind speed around a certain location. Search longitude and latitude from the internet.

        args:
            latitude : latitude value of the location
            longitude : longitude value of the location
    """
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']

In [12]:
!pip install yfinance

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/fa/6f/dba34a52f77ee05490eaff20fec1934f3cf12afaf538f1de1c81367f7dbc/yfinance-0.2.58-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pandas>=1.3.0 from https://files.pythonhosted.org/packages/52/11/9eac327a38834f162b8250aab32a6781339c69afe7574368fffe46387edf/pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (89 kB)
  Obtaining dependency information for multitasking>=0.0.7 from https://files.pythonhosted.org/packages/3e/8a/bb3160e76e844db9e69a413f055818969c8acade64e1a9ac5ce9dfdcf6c1/multitasking-0.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for pytz>=2022.5 from https://files.pythonhosted.org/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for frozendict>=2.3.4 from https://f

In [42]:
# Tool 3: Stock Price 
import yfinance as yf

@tool(parse_docstring=True)
def get_stock_price(ticker: str) -> str:
    '''
        This function is called to get the latest closing stock price of a company. 
        
        Args:
            ticker (str): The stock ticker symbol (e.g., 'AAPL', 'GOOG').

        Returns:
            str: A message with the latest closing stock price, or an error message if data is unavailable.
    '''
    stock = yf.Ticker(ticker)
    data = stock.history(period="1d")
    if data.empty:
        return f"No data found for {ticker}."
    latest_price = data['Close'].iloc[-1]
    return f"The latest closing price of {ticker} is ${latest_price:.2f}"

In [43]:
tools = [search, get_weather, get_stock_price]

## Building The AI Agent with Langchain

In [45]:
model_with_tools = model.bind_tools(tools)

In [46]:
# No Tool Calling
from langchain_core.messages import HumanMessage

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [47]:
# calling weather
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'latitude and longitude of San Francisco'}, 'id': 'call_AjMG7WEBkasGFVeEyULqmFbA', 'type': 'tool_call'}]


In [48]:
from langchain.agents import create_tool_calling_agent
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(model, tools, prompt)

/Users/hizkiafebianto/Documents/GitHub/revou-gen-ai-tutorial/venv/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [49]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [51]:
agent_executor.invoke({"input": "whats the weather in sf?"})

{'input': 'whats the weather in sf?',
 'output': 'The current temperature in San Francisco is 13.2°C with a wind speed of 5.4 m/s.'}

In [52]:
agent_executor.invoke({"input": "Give me the stock price of Tesla"})

{'input': 'Give me the stock price of Tesla',
 'output': 'The latest closing stock price of Tesla (TSLA) is $280.26.'}

In [ ]:
# Calling two tools
for event in agent_executor.stream({"input": "what's the wind speed in Sumber?"}):
    print(event)

{'actions': [ToolAgentAction(tool='tavily_search_results_json', tool_input={'query': 'Sumber latitude and longitude'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'Sumber latitude and longitude'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_S6dm1HEsR4JN7GVHOIJRGWmd', 'function': {'arguments': '{"query":"Sumber latitude and longitude"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'service_tier': 'default'}, id='run--dd507d57-6d2a-4ff4-abd2-fc31be80178f', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Sumber latitude and longitude'}, 'id': 'call_S6dm1HEsR4JN7GVHOIJRGWmd', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'tavily_search_results_json', 'args': '{"query":"Sumber latitude and longitude"}', 'id': 'call_S6dm1HEsR4JN7GVHOIJRGWmd'

## END